In [ ]:
from pyabc import (ABCSMC, Distribution, RV,
                   MultivariateNormalTransition)
import scipy as sp
import scipy.stats as st
import tempfile
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def model(parameter):
    return {"data": parameter["mean"] + sp.randn()}

In [ ]:
prior = Distribution(mean=RV("uniform", 0, 5))

In [ ]:
def distance(x, y):
    return abs(x["data"] - y["data"])

In [ ]:
abc = ABCSMC(model, prior, distance)

In [ ]:
db_path = ("sqlite:///" +
           os.path.join(tempfile.gettempdir(), "test.db"))
observation = 2.5
abc.set_data({"data": observation}, db_path)

In [ ]:
history = abc.run(minimum_epsilon=.2, max_nr_populations=10)

In [ ]:
history is abc.history

In [ ]:
df = pd.DataFrame({"mean": sp.linspace(0, 5)})
kde = MultivariateNormalTransition()
for t in range(history.max_t+1):
    posterior_particles = history.get_distribution(m=0, t=t)
    kde.fit(*posterior_particles)
    df["PDF t={}".format(t)] = kde.pdf(df[["mean"]])

In [ ]:
ax = df.plot(x="mean");
ax.axvline(observation, color="k", linestyle="dashed");